# Proyecto 1 - Analítica de Textos (G16)
- Daniel Arango Cruz ( d.arangoc2 )
- Javier Cerino Pulido ( j.cerino )
- Marco Zuliani Escalante ( m.zuliani )

## Caso

La Organización de las Naciones Unidas (ONU) adopta, el 25 de septiembre del año 2015, la Agenda 2030i para el desarrollo sostenible, cuyo fin es reducir la pobreza, garantizar acceso a la salud y educación, buscar igualdad de género y oportunidades, disminuir el impacto ambiental, entre otros. Esta agenda se basa en 17 objetivos de desarrollo sostenibleii (ODS) y 169 metas (derivadas de los diferentes ODS). 

## Objetivo
El objetivo es desarrollar un modelo de clasificación, con técnicas de aprendizaje automático, que permita relacionar de manera automática un texto según los ODS. Al igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos de planeación participativa para el desarrollo a nivel territorial.

**Dado que somos el grupo 16, se abordarán los objetivos de desarrollo sostenible 6, 7 y 16.**

## 0. Import y configuración de dependencias

In [1]:
import numpy as np
import pandas as pd

from joblib import dump, load

import nltk

import ftfy

import re
from nltk.corpus import stopwords
from num2words import num2words
from nltk.tokenize import word_tokenize
from sklearn import linear_model

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option("display.max_colwidth", None)

### Obtención de las stopwords en español
Palabras que no aportan información al análisis debido a su uso común y en cualquier contexto.

In [4]:
nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("spanish")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Javier\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Javier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

## 1. Carga de datos

Se carga el archivo de datos asociado a los objetivos de desarrollo sostenible 6, 7 y 16.

In [6]:
data = pd.read_excel("data/cat_6716.xlsx")

Se revisa el numero de registros del dataframe

In [7]:
data.shape

(3000, 2)

Se revisan las columnas del dataframe

In [8]:
data.columns

Index(['Textos_espanol', 'sdg'], dtype='object')

Se revisan algunos registros del dataframe

In [9]:
data.head(6)

,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequía, se espera que disminuyan todos los aportes, pero que aumente la demanda de agua de los cultivos y de la vegetación, lo que provocará una reducción del caudal de los ríos que están conectados hidrológicamente con el acuífero, así como una menor disponibilidad de agua para otros ecosistemas dependientes de las aguas subterráneas. Estas externalidades espaciales pueden incluir el agotamiento de los cursos de agua y el descenso del nivel freático local. Por ejemplo, el bombeo continuo del pozo B provocará el agotamiento de los cursos de agua y la reducción de su caudal.",6
1,"Hay una gran cantidad de literatura sobre Australia en esta área que sugiere fuertemente que el comercio de agua, especialmente en los mercados a corto plazo, no solo permite mitigar sustancialmente los impactos de las sequías en la agricultura, sino que ahora los agricultores lo consideran como un medio útil y básico. instrumento en su caja de herramientas de gestión de riesgos. Los desafíos políticos para Australia en el área de la regulación del agua en la agricultura están más en las áreas de cómo ajustar el sistema existente y los problemas de solidez relacionados con el cambio climático. Si bien el ejemplo australiano ilustra las posibles ganancias de eficiencia que surgen del comercio de agua, especialmente para gestionar la escasez de agua en circunstancias de sequía, la elección de un instrumento siempre debe considerar la gama completa de costos y beneficios.",6
2,"Los procesos de descentralización, emprendidos para una serie de actividades que antes realizaba el gobierno central, llevaron a modificaciones importantes en la forma en que se administraba y operaba el sector del agua. Las únicas excepciones a esta tendencia fueron Azerbaiyán, Bielorrusia y Turkmenistán, donde la gestión del sector del agua ha permanecido fuertemente centralizada. En Azerbaiyán, por ejemplo, la empresa nacional AZERSU JSC suministra agua al 65 % de la población, con un 10 % proporcionado por pequeños operadores privados y un 25 % por los propios hogares.",6
3,"Esto puede tener consecuencias sustanciales para la calidad del agua, especialmente en el río Mississippi y el norte del Golfo de México (De la Torre Ugarte et al., Si se requieren aportes de nutrientes para las plantaciones de madera, la infiltración y la escorrentía de nitrógeno también pueden representar un riesgo para las aguas subterráneas). (Lattimore et al., otra conclusión importante es que la ubicación de la producción y el tipo de producción de labranza, el sistema de rotación de cultivos y otras prácticas de gestión agrícola utilizadas en la producción de materias primas para la producción de bioenergía también influirán en gran medida en la calidad del agua (De la Torre Ugarte et al. ,",6
4,"La función de beneficio también incorpora parámetros que afectan a todos los usuarios, como los precios de insumos y productos y el clima.7 Por ejemplo, en un año seco, se espera que el beneficio de aplicar cualquier cantidad de agua a un cultivo sea mayor que en un año seco. año húmedo. La optimización óptima requeriría una valoración explícita de todos los posibles servicios ambientales asociados con el recurso de agua subterránea, tanto ahora como en el futuro. En cambio, la sociedad decide el nivel de servicios hidrológicos que es deseable y el problema económico es lograr ese nivel utilizando una solución que maximice los beneficios para los usuarios del agua.",6
5,"La descentralizaciÃ³n tambiÃ©n implica dar mayor protagonismo a los ComitÃ©s TÃ©cnicos de Aguas SubterrÃ¡neas (COTAS) en la gestiÃ³n de los acuÃferos, fortalecer la organizaciÃ³n y funcionamiento de los Consejos de Cuenca y sus Ã³rganos auxiliares, y consolidar las funciones de gobernanza y organizaciÃ³n regional de la CONAGUA. No existe un modelo Ãºnico para mejorar la gobernanza pÃºblica en el sector hÃdrico y superar los obstÃ¡culos institucionales para su implementaciÃ³n, p